In [ ]:
import os
import json
import glob

# input and output directory
transformed_dir = "../../output/CAMELTrack_outputs/Outdoor/transformed/"
output_dir = os.path.join(transformed_dir, "with_jersey_number")
os.makedirs(output_dir, exist_ok=True)

# illegible.json の読み込み
with open("../../output/jersey-number-pipeline_outputs/OutdoorResults/ID_merging/illegible.json", "r", encoding="utf-8") as f:
    illegible_data = json.load(f).get("illegible", {})
print(f"illegible.json: {len(illegible_data)} images")

# final_results.json の読み込み
with open("../../output/jersey-number-pipeline_outputs/OutdoorResults/ID_merging/final_results.json", "r", encoding="utf-8") as f:
    final_results = json.load(f)
print(f"final_results.json: {len(final_results)} images")

# transformed フォルダ内の *.txt ファイルを取得（with_jersey_numberフォルダは除く）
txt_files = glob.glob(os.path.join(transformed_dir, "*.txt"))

for txt_file in txt_files:
    base_name = os.path.splitext(os.path.basename(txt_file))[0]
    
    # 対象画像の final_results 情報（存在しない場合は空の dict）
    final_result_for_image = final_results.get(base_name, {})
    
    # 対象画像の illegible 情報（除外すべき track リスト、存在しなければ空リスト）
    illegible_tracks = illegible_data.get(base_name, [])
    
    output_lines = []
    with open(txt_file, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            # 行をカンマ区切りで分割（フレーム番号, track_id, x, y）
            parts = line.split(",")
            if len(parts) < 4:
                continue
            frame, track_id, x, y = parts[0], parts[1], parts[2], parts[3]
            track_key = "track" + track_id  # final_results のキーは "trackX" の形式
            
            # illegible に含まれている track は除外
            if track_key in illegible_tracks:
                continue
            
            # final_results に該当 track が存在し、かつジャージ番号が -1 でなければ取得
            jersey_number = final_result_for_image.get(track_key)
            if jersey_number is None or jersey_number == -1 or jersey_number == "-1":
                continue
            
            # 新たな列としてジャージ番号を追加
            new_line = ",".join([frame, track_id, x, y, str(jersey_number)])
            output_lines.append(new_line)
            
    # 出力先ファイルに保存
    output_file = os.path.join(output_dir, os.path.basename(txt_file))
    with open(output_file, "w", encoding="utf-8") as f:
        for line in output_lines:
            f.write(line + "\n")
    print(f"Saved: {output_file}")


illegible.json: 96 images
final_results.json: 97 images
Saved: ../../CAMELTrack_outputs/Outdoor/transformed_no_merging/with_jersey_number/IMG_0104_1.txt
Saved: ../../CAMELTrack_outputs/Outdoor/transformed_no_merging/with_jersey_number/IMG_0104_2.txt
Saved: ../../CAMELTrack_outputs/Outdoor/transformed_no_merging/with_jersey_number/IMG_0104_3.txt
Saved: ../../CAMELTrack_outputs/Outdoor/transformed_no_merging/with_jersey_number/IMG_0104_4.txt
Saved: ../../CAMELTrack_outputs/Outdoor/transformed_no_merging/with_jersey_number/IMG_0104_5.txt
Saved: ../../CAMELTrack_outputs/Outdoor/transformed_no_merging/with_jersey_number/IMG_0104_6.txt
Saved: ../../CAMELTrack_outputs/Outdoor/transformed_no_merging/with_jersey_number/IMG_0107_8.txt
Saved: ../../CAMELTrack_outputs/Outdoor/transformed_no_merging/with_jersey_number/IMG_0110_4.txt
Saved: ../../CAMELTrack_outputs/Outdoor/transformed_no_merging/with_jersey_number/IMG_0107_9.txt
Saved: ../../CAMELTrack_outputs/Outdoor/transformed_no_merging/with_jer

In [ ]:
import os
import glob
import math
import numpy as np
from scipy.spatial.distance import jensenshannon

def compute_js(p, q, eps=1e-10):
    p = np.asarray(p, dtype=np.float64)
    q = np.asarray(q, dtype=np.float64)
    p /= (p.sum() + eps)
    q /= (q.sum() + eps)
    return jensenshannon(p, q)

# ------------------------------
# 1. 入出力ディレクトリ設定
# ------------------------------
jersey_dir = "../../output/CAMELTrack_outputs/Outdoor/transformed_no_merging/with_jersey_number/"
output_dir = os.path.join(jersey_dir, "with_team")
os.makedirs(output_dir, exist_ok=True)

free_throw_gt_dir = "../../output/ground_truth/Outdoor/MOT_files/split_transformed/free_throw"
check_ball_gt_dir = "../../output/ground_truth/Outdoor/MOT_files/split_transformed/check_ball"

# 全ファイルを処理
for file_path in glob.glob(os.path.join(jersey_dir, "*.txt")):
    basename = os.path.basename(file_path)
    print("----- Processing file:", basename, "-----")

    # free_throw / check_ball 判定
    if os.path.exists(os.path.join(free_throw_gt_dir, basename)):
        designated_point = (752.5, 580)
        is_free_thorw = True
    elif os.path.exists(os.path.join(check_ball_gt_dir, basename)):
        designated_point = (752.5, 1105)
        is_free_thorw = False
    else:
        print("-> Ground truth not found, skipping.")
        continue

    # identifier 抽出
    if basename.startswith("IMG_") and basename.endswith(".txt"):
        identifier = basename[len("IMG_"):-4]
    else:
        identifier = os.path.splitext(basename)[0]

    # ------------------------------
    # 2. ファイル読み込み & フレームごとにグループ化
    # ------------------------------
    records_by_frame = {}
    with open(file_path, encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split(",")
            if len(parts) < 5: 
                continue
            frame = int(parts[0])
            rec = {
                "frame": frame + 1,
                "track": parts[1],
                "x": float(parts[2]),
                "y": float(parts[3]),
                "jersey": parts[4]
            }
            records_by_frame.setdefault(frame, []).append(rec)

    frames = sorted(records_by_frame)
    if not frames:
        print("-> No frames found, skipping.")
        continue

    # ------------------------------
    # 2-1. 初期割り当て
    # ------------------------------
    track_assignment = {}
    
    # 各フレームを見ていき、トラックIDごとに１件ずつ初期レコードを貯める
    init_records = {}  # key: track, value: rec(dict)
    for frame in frames:
        for rec in records_by_frame[frame]:
            tid = rec["track"]
            # まだ登録されていないトラックなら採用（ここを最新のものにするなら常に上書き）
            if tid not in init_records:
                # フレーム番号は＋1しておく
                rec["frame"] = frame + 1
                init_records[tid] = rec
        # トラック数が6つ揃ったらループを抜けて割り当てへ
        if len(init_records) >= 6:
            break
    
    first_frame = frames[0]
    first_frame_records = records_by_frame[first_frame]
    print(f"-> First frame (#{first_frame}) has {len(first_frame_records)} tracks.")
    
    first_records = list(init_records.values())

    # 距離計算
    for rec in first_records:
        rec["dist"] = math.hypot(rec["x"]-designated_point[0], rec["y"]-designated_point[1])

    # free_throw(check_ball=False) / check_ball(check_ball=True) で分岐
    hist_dir = os.path.join("../../color_histograms/Outdoor/non_ID_merging/", identifier)

    if is_free_thorw:
        # free_throw: 一番近い track を offense
        off = min(first_records, key=lambda r: r["dist"])
        track_assignment[off["track"]] = "Offense"
        print("free_throw: initial offense ->", off["track"])

        # offense ヒストグラムで JS divergence 小さいものをあと2名 offense
        off_hist_path = os.path.join(hist_dir, f"{identifier}_track{off['track']}.npy")
        if os.path.exists(off_hist_path):
            off_hist = np.load(off_hist_path)
            divergences = []
            for rec in first_records:
                tid = rec["track"]
                if tid == off["track"]:
                    continue
                path = os.path.join(hist_dir, f"{identifier}_track{tid}.npy")
                if os.path.exists(path):
                    divergences.append((tid, compute_js(off_hist, np.load(path))))
            divergences.sort(key=lambda x: x[1])
            for tid, d in divergences[:2]:
                track_assignment[tid] = "Offense"
                print(f"  + offense by JS: {tid} (d={d:.3f})")
        else:
            print("WARNING: offense histogram missing, skipping JS step.")

    else:
        # check_ball(check_ball): 最初に offense, 次に offense に最も近い track を defense
        off = min(first_records, key=lambda r: r["dist"])
        track_assignment[off["track"]] = "Offense"
        print("check_ball: initial offense ->", off["track"])

        # defense candidate
        remaining = [r for r in first_records if r["track"] != off["track"]]
        def_cand = min(remaining, key=lambda r: math.hypot(r["x"]-off["x"], r["y"]-off["y"]))
        track_assignment[def_cand["track"]] = "Defense"
        print("check_ball: initial defense ->", def_cand["track"])

        # JS divergence による追加割り当て（offense→2名, defense→2名）
        off_hist_path = os.path.join(hist_dir, f"{identifier}_track{off['track']}.npy")
        def_hist_path = os.path.join(hist_dir, f"{identifier}_track{def_cand['track']}.npy")
        # offense JS
        if os.path.exists(off_hist_path):
            off_hist = np.load(off_hist_path)
            offs = []
            for rec in remaining:
                tid = rec["track"]
                if tid in track_assignment: continue
                path = os.path.join(hist_dir, f"{identifier}_track{tid}.npy")
                if os.path.exists(path):
                    offs.append((tid, compute_js(off_hist, np.load(path))))
            offs.sort(key=lambda x: x[1])
            for tid, d in offs[:2]:
                track_assignment[tid] = "Offense"
                print(f"  + offense by JS: {tid} (d={d:.3f})")
        else:
            print("WARNING: offense histogram missing.")

        # defense JS
        if os.path.exists(def_hist_path):
            def_hist = np.load(def_hist_path)
            defs = []
            for rec in remaining:
                tid = rec["track"]
                if tid in track_assignment: continue
                path = os.path.join(hist_dir, f"{identifier}_track{tid}.npy")
                if os.path.exists(path):
                    defs.append((tid, compute_js(def_hist, np.load(path))))
            defs.sort(key=lambda x: x[1])
            for tid, d in defs[:2]:
                track_assignment[tid] = "Defense"
                print(f"  + defense by JS: {tid} (d={d:.3f})")
        else:
            print("WARNING: defense histogram missing.")

    # 残りを Defense
    for rec in first_records:
        tid = rec["track"]
        if tid not in track_assignment:
            track_assignment[tid] = "Defense"
    print("-> First frame assignments:", track_assignment)


    
    # ------------------------------
    # 2.5. 各トラックの座標を補完・外挿（修正版）
    # ------------------------------
    # 全トラック一覧
    all_tracks = set(track_assignment.keys())
    min_frame, max_frame = frames[0], frames[-1]

    # トラックごとにフレーム→(x,y,jersey,role) を整理
    track_positions = {tid: {} for tid in all_tracks}
    for frame in frames:
        for rec in records_by_frame[frame]:
            tid = rec["track"]
            if tid in all_tracks:
                role = track_assignment[tid]
                track_positions[tid][frame] = (
                    rec["x"], rec["y"], rec["jersey"], role
                )

    # 線形補完・外挿後の新レコード辞書を用意
    new_records_by_frame = {f: [] for f in range(min_frame, max_frame + 1)}

    for tid, pos_dict in track_positions.items():
        exist_fs = sorted(pos_dict.keys())
        if len(exist_fs) < 2:
            # 補完・外挿には最低2点必要
            continue

        # 先頭2フレーム差分（外挿用）
        f0, f1 = exist_fs[0], exist_fs[1]
        x0, y0, jersey0, role0 = pos_dict[f0]
        x1, y1, _, _ = pos_dict[f1]
        delta_start = ((x1 - x0) / (f1 - f0), (y1 - y0) / (f1 - f0))

        # 最終2フレーム差分（外挿用）
        fN_1, fN = exist_fs[-2], exist_fs[-1]
        xN_1, yN_1, _, _ = pos_dict[fN_1]
        xN, yN, jerseyN, roleN = pos_dict[fN]
        delta_end = ((xN - xN_1) / (fN - fN_1), (yN - yN_1) / (fN - fN_1))

        # 全フレームループ
        for f in range(min_frame, max_frame + 1):
            if f in pos_dict:
                x, y, jersey, role = pos_dict[f]
            else:
                if f < f0:
                    # 開始前外挿
                    x = x0 + delta_start[0] * (f - f0)
                    y = y0 + delta_start[1] * (f - f0)
                    jersey, role = jersey0, role0
                elif f > fN:
                    # 終了後外挿
                    x = xN + delta_end[0] * (f - fN)
                    y = yN + delta_end[1] * (f - fN)
                    jersey, role = jerseyN, roleN
                else:
                    # 中間線形補完
                    prev_f = max([ef for ef in exist_fs if ef < f])
                    next_f = min([ef for ef in exist_fs if ef > f])
                    x_prev, y_prev, _, _ = pos_dict[prev_f]
                    x_next, y_next, _, _ = pos_dict[next_f]
                    ratio = (f - prev_f) / (next_f - prev_f)
                    x = x_prev + (x_next - x_prev) * ratio
                    y = y_prev + (y_next - y_prev) * ratio
                    jersey, role = pos_dict[prev_f][2], pos_dict[prev_f][3]

            new_records_by_frame[f].append({
                "frame": f,
                "track": tid,
                "x": x,
                "y": y,
                "jersey": jersey,
                "role": role
            })

    # 置き換え
    records_by_frame = new_records_by_frame
    frames = sorted(records_by_frame.keys())
    
    
    # ------------------------------
    # 5. 各トラックの出現期間・フレーム数を記録（画面内にいるフレームのみカウント）
    # ------------------------------
    track_info = {}
    for frame in frames:
        for rec in records_by_frame[frame]:
            x, y = rec["x"], rec["y"]
            # 画面内判定: 0<=x<=1505 and 0<=y<=1105 のときだけカウント
            if not (0 <= x <= 1505 and 0 <= y <= 1105):
                continue

            tid = rec["track"]
            role = track_assignment.get(tid, "")
            if tid not in track_info:
                track_info[tid] = {
                    "start": frame,
                    "end": frame,
                    "count": 1,
                    "jersey": rec["jersey"],
                    "role": role
                }
            else:
                track_info[tid]["end"] = frame
                track_info[tid]["count"] += 1

    # ------------------------------
    # 6. 各フレーム内で検出トラック数を最大6に調整
    # ------------------------------
    for frame in frames:
        recs = records_by_frame[frame]
        # 総検出数が6以下ならそのまま
        if len(recs) <= 6:
            continue

        # track_info の出現回数順（多い順）に並べ替え、上位6の track ID をキープ
        # 同じ track が複数レコードあるなら、それも含めて全てキープ
        sorted_tids = sorted(
            {r["track"] for r in recs},
            key=lambda t: track_info.get(t, {}).get("count", 0),
            reverse=True
        )
        keep_tids = set(sorted_tids[:6])

        # フレーム内レコードを絞り込み
        filtered = [r for r in recs if r["track"] in keep_tids]
        records_by_frame[frame] = filtered

        print(f"Frame {frame}: total tracks reduced from {len(recs)} to {len(filtered)}")



    # ------------------------------
    # 7. 出力用に各レコードの背番号を更新（チーム情報の接頭辞付与）
    # ------------------------------
    for frame in frames:
        for rec in records_by_frame[frame]:
            tid = rec["track"]
            role = track_assignment.get(tid, "")
            jersey = track_info.get(tid, {}).get("jersey", rec["jersey"])
            if role == "Offense":
                rec["jersey"] = "O" + str(jersey)
            elif role == "Defense":
                rec["jersey"] = "D" + str(jersey)
            else:
                rec["jersey"] = str(jersey)
    
    # ------------------------------
    # 8. ファイル出力：各行は "frame,track,x,y,統合背番号" の形式
    # ------------------------------
    output_lines = []
    for frame in frames:
        for rec in records_by_frame[frame]:
            line = f'{rec["frame"]},{rec["track"]},{rec["x"]},{rec["y"]},{rec["jersey"]}'
            output_lines.append(line)
    
    out_path = os.path.join(output_dir, basename)
    with open(out_path, "w", encoding="utf-8") as f:
        for line in output_lines:
            f.write(line + "\n")
    print("-> Output file written to:", out_path, "\n")


----- Processing file: IMG_0104_1.txt -----
-> First frame (#1) has 6 tracks.
check_ball: initial offense -> 1
check_ball: initial defense -> 3
  + offense by JS: 7 (d=0.172)
  + offense by JS: 2 (d=0.231)
  + defense by JS: 4 (d=0.238)
  + defense by JS: 8 (d=0.349)
-> First frame assignments: {'1': 'Offense', '3': 'Defense', '7': 'Offense', '2': 'Offense', '4': 'Defense', '8': 'Defense'}
-> Output file written to: ../../CAMELTrack_outputs/Outdoor/transformed_no_merging/with_jersey_number/with_team/IMG_0104_1.txt 

----- Processing file: IMG_0104_2.txt -----
-> First frame (#1) has 6 tracks.
free_throw: initial offense -> 3
  + offense by JS: 9 (d=0.358)
  + offense by JS: 6 (d=0.377)
-> First frame assignments: {'3': 'Offense', '9': 'Offense', '6': 'Offense', '1': 'Defense', '10': 'Defense', '2': 'Defense'}
-> Output file written to: ../../CAMELTrack_outputs/Outdoor/transformed_no_merging/with_jersey_number/with_team/IMG_0104_2.txt 

----- Processing file: IMG_0104_3.txt -----
-> Fir